In [24]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = "u1J0sXUfmB0"
languages = ['en']
transcript = YouTubeTranscriptApi().list(video_id)\
    .find_transcript(languages)\
    .fetch(preserve_formatting=True)

In [25]:
for snippet in transcript.snippets[:5]:
    print(f"  [{snippet.start:.2f}s - {snippet.start + snippet.duration:.2f}s]: {snippet.text}")


  [0.00s - 3.00s]: So,
  [3.60s - 8.00s]: let me try and catch you up uh uh since
  [5.60s - 10.24s]: we've been gone. Uh the economy hit a
  [8.00s - 13.68s]: brick wall, stalling on jobs while
  [10.24s - 17.12s]: inflation continued to rise. So, Trump


In [30]:
text = ""
for snippet in transcript.snippets:
    text += snippet.text + " "

print(text)

So, let me try and catch you up uh uh since we've been gone. Uh the economy hit a brick wall, stalling on jobs while inflation continued to rise. So, Trump met with Putin in Alaska to force Ukraine to accept Crackle Barrel's new logo while Leubu released a new Epstein file and Taylor got engaged. It's a blur. It's all a blur, but all of it takes a backseat to continuing concerns over the president's health. We haven't really seen much of the president over the last week. Basically absent from the public eye. He's clearly hiding something about his health. The hashtag Trump is dead trended on social media. What the you people, you reporters have no chill. guy can't take a few days for some RNR and a non-surgical breast production without everybody [Applause] suddenly pulling out the toe tags on the president. It does say something about the ubiquity of Donald Trump in our lives that we don't hear from him for 20 minutes and we're like, he's dead. I think he's check can we track him? It'

In [27]:
import spacy
from collections import Counter
nlp = spacy.load("en_core_web_sm")

complete_doc = nlp(text)

words = [
     token.text
     for token in complete_doc
     if not token.is_stop and not token.is_punct
]

print(Counter(words).most_common(5))

[('Trump', 60), ('Applause', 32), ('like', 29), ('know', 29), ('president', 24)]


get general meaning or keyworkds

In [28]:
import textacy

en = textacy.load_spacy_lang("en_core_web_sm", disable=("parser",))
doc = textacy.make_spacy_doc(text, lang=en)
doc._.preview

doc = textacy.make_spacy_doc((text), lang="en_core_web_sm")

# list(textacy.extract.ngrams(doc, 5, filter_punct=True))

textacy.extract.keyterms.textrank(doc, normalize="lemma", topn=100)


# textacy.extract.keyterms.sgrank(doc, ngrams=(1, 2, 3, 4), normalize="lower", topn=0.1)


[('time Trump', 0.006561891528237518),
 ('Trump signature', 0.006490652767522205),
 ('Donald Trump', 0.005929326568426114),
 ('President Trump', 0.00585176055032369),
 ('Trump book', 0.005524214980010635),
 ('U Trump', 0.005080488810040782),
 ('Eric Trump', 0.005079759573222604),
 ('Epstein birthday book', 0.004291508224679019),
 ('birthday book signature', 0.004088776151835201),
 ('sure people', 0.00373583890512578),
 ('scared town people', 0.003711182634077901),
 ('man baby president', 0.0036649848315191283),
 ('new Epstein file', 0.0035925428420266856),
 ('young people', 0.0035458445525630355),
 ('past president', 0.0033999260475107186),
 ('year federal prison sentence', 0.003177381115113818),
 ('current big guy', 0.003155235961448828),
 ('wish thing', 0.0031067894803993006),
 ('honorary United States Marshall Service badge', 0.0030439090942500522),
 ('future president', 0.0030342939562392787),
 ('certain thing', 0.0029835932220212166),
 ('good sign', 0.002908544224224873),
 ('nice 

get phrasal verbs with phrasemachine

In [34]:
# import phrasemachine

# nlp = spacy.load("en_core_web_sm")

# doc = nlp(text)

# # Extract tokens and POS tags
# tokens = [token.text for token in doc]
# pos_tags = [token.pos_ for token in doc]

# # Get phrases using Phrasemachine
# phrases_output = phrasemachine.get_phrases(tokens=tokens, postags=pos_tags, output="token_spans")

# print(phrases_output['token_spans'])

import spacy
import phrasemachine
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
tokens = [token.text for token in doc]
pos = [token.pos_ for token in doc]
# print(tokens)
# ['Barack', 'Obama', 'supports', 'expanding', 'social', 'security', '.']
# >>> print(pos)
# ['PROPN', 'PROPN', 'VERB', 'VERB', 'ADJ', 'NOUN', 'PUNCT']
phrasemachine.get_phrases(tokens=tokens, postags=pos)
# {'num_tokens': 7, 'counts': Counter({'barack obama': 1, 'social security': 1})}

{'num_tokens': 7283,
 'counts': Counter({'donald trump': 10,
          'birthday card': 7,
          'department of war': 7,
          'white house': 6,
          'president trump': 6,
          'jeffrey epstein': 6,
          'department of defense': 6,
          'joe biden': 5,
          'political violence': 5,
          'wish kid': 4,
          'supreme court': 3,
          'american history': 3,
          'sexual secret': 3,
          'new york': 3,
          'world war': 3,
          'national guard': 3,
          'charlie kirk': 3,
          'last thing': 3,
          'chronic venus': 2,
          'chronic venus insufficiency': 2,
          'venus insufficiency': 2,
          'menu on a menu': 2,
          'milton burl': 2,
          'mr. president': 2,
          'united states': 2,
          'fragile ego': 2,
          'bravest boy': 2,
          'nobel prize': 2,
          'twilight zone': 2,
          'bad man': 2,
          'more years': 2,
          'dark sexual secret': 2,

get phrasal verbs with stanza

In [37]:
import stanza

# Download English model once (if not already downloaded)
stanza.download('en')

# Build pipeline with needed processors
nlp = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma,depparse')

#text = "They decided to call off the meeting but then the problem blew up quickly."

doc = nlp(text)

for sentence in doc.sentences:
    for word in sentence.words:
        if word.pos == 'VERB':
            # Collect particles or adpositions linked to the verb
            particles = [w.text for w in sentence.words if w.head == word.id and w.deprel in ('compound:prt', 'fixed', 'advmod', 'case')]
            if particles:
                phrasal_verb = word.text + " " + " ".join(particles)
                print(f"Phrasal verb found: {phrasal_verb}, whole sentence: {' '.join([w.text for w in sentence.words])}")


2025-09-15 13:28:02 INFO: Downloaded file to /Users/denismasterov/stanza_resources/resources.json
2025-09-15 13:28:02 INFO: Downloading default packages for language: en (English) ...
2025-09-15 13:28:03 INFO: File exists: /Users/denismasterov/stanza_resources/en/default.zip
2025-09-15 13:28:05 INFO: Finished downloading models and saved to /Users/denismasterov/stanza_resources
2025-09-15 13:28:05 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-09-15 13:28:05 INFO: Downloaded file to /Users/denismasterov/stanza_resources/resources.json
2025-09-15 13:28:06 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charl

Phrasal verb found: catch up, whole sentence: So , let me try and catch you up uh uh since we 've been gone .
Phrasal verb found: met So, whole sentence: So , Trump met with Putin in Alaska to force Ukraine to accept Crackle Barrel 's new logo while Leubu released a new Epstein file and Taylor got engaged .
Phrasal verb found: seen n't really, whole sentence: We have n't really seen much of the president over the last week .
Phrasal verb found: hiding clearly, whole sentence: He 's clearly hiding something about his health .
Phrasal verb found: trended dead, whole sentence: The hashtag Trump is dead trended on social media .
Phrasal verb found: take n't, whole sentence: guy ca n't take a few days for some RNR and a non-surgical breast production without everybody [ Applause ] suddenly pulling out the toe tags on the president .
Phrasal verb found: pulling suddenly out, whole sentence: guy ca n't take a few days for some RNR and a non-surgical breast production without everybody [ Appla

In [50]:
import spacy
from cefrpy import CEFRAnalyzer

# Initialize tools
nlp = spacy.load("en_core_web_sm")
analyzer = CEFRAnalyzer()

def word_cefr_level(word):
    # Auto POS tagging with spaCy to get POS tag (simplified to first two letters)
    doc = nlp(word)
    pos = doc[0].pos_  # e.g. 'NOUN', 'VERB'
    if pos == "NOUN":
        pos_tag = "NN"
    elif pos == "VERB":
        pos_tag = "VB"
    elif pos == "ADJ":
        pos_tag = "JJ"
    elif pos == "ADV":
        pos_tag = "RB"
    else:  # fallback
        return 0
    # Get CEFR level in float
    return analyzer.get_word_pos_level_float(word.lower(), pos_tag)

def filter_text_by_cefr(text, min_level=4.0, max_level = 5.0):  # B1 corresponds roughly to 4.0 float
    doc = nlp(text)
    result_sentences = []
    for sent in doc.sents:
        for token in sent:
            level = word_cefr_level(token.text)
            if level and level >= min_level:# and level <= max_level: 
                result_sentences.append((token.text, level, sent.text))
                # break
    return result_sentences

sentences_with_complex_words = filter_text_by_cefr(text, min_level=4.1, max_level=5.0)
for s in sentences_with_complex_words:
    print(s)


('Crackle', 6.0, "So, Trump met with Putin in Alaska to force Ukraine to accept Crackle Barrel's new logo while Leubu released a new Epstein file and Taylor got engaged.")
('ubiquity', 6.0, "It does say something about the ubiquity of Donald Trump in our lives that we don't hear from him for 20 minutes and we're like, he's dead.")
('definitively', 6.0, 'Although I definitively would not go so far as to say uh alive and kicking.')
('insufficiency', 5.74, 'New today, the White House revealing President Trump has been diagnosed with a condition called chronic venus insufficiency.')
('insufficiency', 5.74, 'Chronic Venus insufficiency.')
('makeup', 5.6, 'These images of the president have drawn attention in recent days, showing apparent bruising on his hands that was covered with makeup.')
('makeup', 5.6, "And what's with the makeup?")
('leprosy', 6.0, "It's not like you can treat leprosy at Sephora.")
('grimace', 6.0, "For all we know, the guy's gone full grimace.")
('puffy', 6.0, 'Blotch

In [2]:
import pandas as pd
from minio import Minio
import re

video_id = "ddk2qOdulWY"
accum = 0

data = []
# Emulating your snippet processing loop
for snippet in transcript.snippets:  # change this to your real snippet list
    id_snippet = f"{video_id}_{int(round(snippet.start, 2) * 100)}"
    arr = snippet.text.split(' ')
    for token in arr:
        accum += 1
        clean_token = re.sub(r'[^a-zA-Z0-9]', '', token).lower()
        data.append({
            "snippet_id": id_snippet,
            "order_num": accum,
            "value": clean_token,
            "video_id": video_id,
            "start_ts": round(snippet.start, 2),
            "end_ts": round(snippet.start + snippet.duration, 2)
        })

# Convert to DataFrame
df = pd.DataFrame(data)

In [34]:
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyarrow-21.0.0-cp39-cp39-macosx_12_0_arm64.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 12.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
df['snippet_id'] = df['snippet_id'].astype(str)
df['value'] = df['value'].astype(str)
df['video_id'] = df['video_id'].astype(str)

In [13]:
import pyarrow as pa

parquet_file_path = f"{video_id}.parquet"
# Initialize MinIO client (adjust endpoint, access/secret keys)
minio_client = Minio(
    "localhost:9000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False
)

bucket_name = "input"
object_name = f"youtube/{parquet_file_path}"

# Upload file
minio_client.fput_object(
    bucket_name,
    object_name,
    parquet_file_path
)


df.to_parquet(parquet_file_path, engine='pyarrow', index=False)
print(f"Saved parquet file: {parquet_file_path}")

print(f"Uploaded {parquet_file_path} to MinIO bucket '{bucket_name}' as '{object_name}'")


Saved parquet file: ddk2qOdulWY.parquet
Uploaded ddk2qOdulWY.parquet to MinIO bucket 'input' as 'youtube/ddk2qOdulWY.parquet'


In [14]:
bucket_name = "input"
object_name = f"youtube/{video_id}.parquet"

# Get the object as a stream
data_stream = minio_client.get_object(bucket_name, object_name)

df = pd.read_parquet(parquet_file_path, engine='pyarrow')


In [15]:
# display count per value field wih order descending by number of occurences
value_counts = df['value'].value_counts().reset_index()

In [24]:
# display mode all for pandas dataframe
# sort by field length of value field
value_counts['length'] = value_counts['value'].apply(len) 
value_counts.sort_values(by=['length'], ascending=False, inplace=True)
value_counts

,value,count,length
293,characteristics,1,15
252,alternatively,1,13
181,interpolation,1,13
136,partitioning,1,12
121,specifically,1,12
...,...,...,...
272,us,1,2
7,so,14,2
8,is,13,2
10,a,10,1
